In [77]:
import pandas as pd
from datetime import datetime

In [63]:
wetterdaten = pd.read_csv("../Temp_DWH/tageswetterdaten.csv")
geodaten = pd.read_csv("../Temp_DWH/geographie.csv")
daxdaten = pd.read_csv("../Temp_DWH/DAXHISTORIE.csv")
gastrodaten =pd.read_csv("../Temp_DWH/Gastronomieumsaetze_flat_20240623_134949.csv")
wetterdaten.drop(columns=["Unnamed: 0"])
geodaten.drop(columns=["Unnamed: 0"])
daxdaten = daxdaten.rename(columns={"Date":"Datum"})

In [64]:
wetterdaten.Datum = pd.to_datetime(wetterdaten.Datum)
daxdaten.Datum = pd.to_datetime(daxdaten.Datum)

In [65]:
month_mapping = {
    'Januar': 1,
    'Februar': 2,
    'März': 3,
    'April': 4,
    'Mai': 5,
    'Juni': 6,
    'Juli': 7,
    'August': 8,
    'September': 9,
    'Oktober': 10,
    'November': 11,
    'Dezember': 12
}

# String to Zahl für späteres Merge
gastrodaten['Monat'] = gastrodaten['Monat'].map(month_mapping)

In [66]:
gastrodaten['day'] = 1
gastrodaten = gastrodaten.rename(columns={"Monat" : "month", "Jahr" : "year"})
gastrodaten['Datum'] = pd.to_datetime(gastrodaten[['year', 'month',"day"]])
# löschen altes Datumsformats
dr_columns = ["month","year","day"]
gastrodaten = gastrodaten.drop(columns=dr_columns)
gastrodaten

,Unternehmen_Id,Unternehmensbeschreibung,Umsatz in millionen,Datum
0,WZ08-55,Beherbergung,92.3,1994-01-01
1,WZ08-551,"Hotels, Gasthöfe und Pensionen",91.9,1994-01-01
2,WZ08-552,Ferienunterkünfte u. ähnliche Beherbergungsstä...,97.3,1994-01-01
3,WZ08-553,Campingplätze,68.4,1994-01-01
4,WZ08-559,Sonstige Beherbergungsstätten,127.1,1994-01-01
...,...,...,...,...
3966,WZ08-561,"Restaurants, Gaststätten, Imbissstuben, Cafes ...",76.6,2024-01-01
3967,WZ08-562,Caterer und sonstige Verpflegungsdienstleistungen,87.4,2024-01-01
3968,WZ08-563,Ausschank von Getränken,52.0,2024-01-01
3969,WZ08-561-01,Gaststättengewerbe,74.1,2024-01-01


In [81]:
current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')

In [82]:
merge_geo_wetter = pd.merge(geodaten, wetterdaten, on="stations_id")
merge_dax = pd.merge(merge_geo_wetter, daxdaten, on="Datum")
merge_all = pd.merge(merge_dax, gastrodaten, on="Datum")
merge_all = merge_all.drop(columns=["Unnamed: 0_x", "Unnamed: 0_y"])
merge_all = merge_all.rename(columns={"geogr.breite" : "geo_breite", "geogr.laenge" : "geo_laenge"})
merge_all.to_csv(f"../Data_Lake/all_{current_datetime}.csv", index=False)
merge_all.to_csv(f"../Temp_DWH/gemerged.csv", index=False)	

In [74]:
for column in merge_all.columns:
    print(f"Spalte: {column}")

Spalte: Unnamed: 0_x
Spalte: stationsname
Spalte: geogr.breite
Spalte: geogr.laenge
Spalte: stationshoehe
Spalte: stations_id
Spalte: Unnamed: 0_y
Spalte: Datum
Spalte: QN_3
Spalte: FX
Spalte: FM
Spalte: QN_4
Spalte: RSK
Spalte: RSKF
Spalte: SDK
Spalte: SHK_TAG
Spalte: NM
Spalte: VPM
Spalte: PM
Spalte: TMK
Spalte: UPM
Spalte: TXK
Spalte: TNK
Spalte: TGK
Spalte: eor
Spalte: Open
Spalte: High
Spalte: Low
Spalte: Close
Spalte: Adj Close
Spalte: Volume
Spalte: Unternehmen_Id
Spalte: Unternehmensbeschreibung
Spalte: Umsatz in millionen
